In [1]:
# !pip install mediapipe

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import cv2
import mediapipe as mp
import numpy as np
import os
import pandas as pd
from PIL import Image
import numpy as np
import pandas as pd

In [4]:
# Kích thướng mỗi frame
IMAGE_WIDTH,  IMAGE_HEIGHT = 160, 120
# số lượng frame sẽ lấy ở mỗi video
SEQUENCE_LENGTH = 20
# Những hành động sẽ nhận dạng
CLASSES_LIST = [
    'Your', 'Love', 'Sleep', 'Name',
    'What', 'Read', 'I', 'Bye', 'You', 'Eat',
    'Hello']

# Link thư mục chứa video train
# DATASET_DIR = './drive/MyDrive/PBL5/DaPhanLoai/'
DATASET_DIR = './DaPhanLoai'


In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands


def frames_extraction(video_path):
    '''
    This function will extract the required frames from a video after resizing and normalizing them.
    Args:
        video_path: The path of the video in the disk, whose frames are to be extracted.
    Returns:
        frames_list: A list containing the resized and normalized frames of the video.
    '''

    video_reader = cv2.VideoCapture(video_path)
    video_frames_count = 0
    s, f = video_reader.read()
    while s:
        s, f = video_reader.read()
        video_frames_count += 1
    video_reader.release()

    # ĐỌc video

    # Calculate the the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

    # Danh sách chứa các frame sẽ lấy
    frames_list = []
    origin_frames_list = []

    # Iterate through the Video Frames.
    with mp_hands.Hands(
            static_image_mode=True,
            max_num_hands=2,
            min_detection_confidence=0.5) as hands:
        with mp_pose.Pose(
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5) as pose:

            cap = cv2.VideoCapture(video_path)
            for frame_counter in range(SEQUENCE_LENGTH):
                cap.set(cv2.CAP_PROP_POS_FRAMES,
                        frame_counter * skip_frames_window)
                success, image = cap.read()

                if not success:
                    cap = cv2.VideoCapture(video_path)
                    continue
                    # break

                # resize image to half its original size
                image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))

                # To improve performance, optionally mark the image as not writeable to
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                hand_results = hands.process(image)
                pose_results = pose.process(image)

                image2 = np.zeros(
                    (image.shape[0], image.shape[1], 3), np.uint8)

                # # Draw the hand annotation on the image.
                if hand_results.multi_hand_landmarks:
                    for hand_landmarks in hand_results.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(
                            image2,
                            hand_landmarks,
                            mp_hands.HAND_CONNECTIONS,
                            mp_drawing_styles.get_default_hand_landmarks_style(),
                            mp_drawing_styles.get_default_hand_connections_style())

                # Draw the pose annotation on the image.
                image.flags.writeable = False
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                mp_drawing.draw_landmarks(
                    image2,
                    pose_results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

                # Append to list feature

                image.flags.writeable = True
                
                frames_list.append(cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)/255)
                origin_frames_list.append(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)/255)

                # resized_frame = cv2.resize(image2, (IMAGE_WIDTH, IMAGE_HEIGHT))
                # frames_list.append(np.any(resized_frame != 0, axis=-1).astype(int))

                # Flip the image2 horizontally for a selfie-view display.
                cv2.imshow('MediaPipe Pose', cv2.flip(image2, 1))
                if cv2.waitKey(1) == ord('e'):
                    return

    # Release the VideoCapture object.
    # Return the frames list.
    return frames_list, origin_frames_list


In [6]:
frames = frames_extraction(f'{DATASET_DIR}/Bye/076.avi')
cv2.destroyAllWindows()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/nhamcotdo/.local/lib/python3.10/site-packages/cv2/qt/plugins"


In [7]:
np.shape(frames[1])

(20, 120, 160)

In [8]:
def create_dataset():
    features = []
    labels = []
    origin_features = []

    video_files_paths = []
    for class_index, class_name in enumerate(CLASSES_LIST):
        feature = []
        origin_feature = []
        label = []
        video_files_path = []

        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')

        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))

        # Iterate through all the files present in the files list.
        for file_name in files_list:
            # print(file_name)
            try:
                # Get the complete video path.
                video_file_path = os.path.join(
                    DATASET_DIR, class_name, file_name)

                # Extract the frames of the video file.
                frames = frames_extraction(video_file_path)

                # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
                # So ignore the vides having frames less than the SEQUENCE_LENGTH.
                if len(frames[0]) == SEQUENCE_LENGTH:

                    # Append the data to their repective lists.
                    # feature.append(frames[0])
                    # origin_feature.append(frames[1])
                    features.append(frames[0])
                    origin_features.append(frames[1])
                    # label.append(class_index)
                    labels.append(class_index)
                    # video_files_path.append(video_file_path)
                    video_files_paths.append(video_file_path)
            except:
                continue

        cv2.destroyAllWindows()

        # Converting the list to numpy arrays
        # feature = np.asarray(feature)
        # origin_feature = np.asarray(origin_feature)
        # label = np.array(label)
        # np.savez_compressed(f'{DATASET_DIR}/{class_name}.npz', features=feature,
        #                     origin_features=origin_feature, labels=label, video_files_paths=video_files_path)

    features = np.asarray(features)
    origin_features = np.asarray(origin_features)
    labels = np.array(labels)
    # Return the frames, class index, and video file path.
    return features, origin_features, labels, video_files_paths


In [9]:
features, origin_features, labels, video_files_paths = create_dataset()

Extracting Data of Class: Your
Extracting Data of Class: Love
Extracting Data of Class: Sleep
Extracting Data of Class: Name
Extracting Data of Class: What
Extracting Data of Class: Read
Extracting Data of Class: I
Extracting Data of Class: Bye
Extracting Data of Class: You
Extracting Data of Class: Eat
Extracting Data of Class: Hello


In [10]:
#import pickle
#with open('data_frame.pickle', 'w+') as f:
    #pickle.dump({'features': features.tolist(), 'labels': labels, 'video_files_paths': video_files_paths}, f, protocol=pickle.HIGHEST_PROTOCOL)


In [11]:
np.savez_compressed(f'{DATASET_DIR}/data.npz', features=features, origin_features=origin_features,
                    labels=labels, video_files_paths=video_files_paths)


In [12]:
dataset = np.load(f'{DATASET_DIR}/data.npz')

In [13]:
video_reader = cv2.VideoCapture('/home/nhamcotdo/Downloads/train-20230322T100607Z-001/train/video750388.avi')
video_frames_count = 0
s, f = video_reader.read()
while s:
    s, f = video_reader.read()
    video_frames_count += 1
video_reader.release()
video_frames_count

0

In [14]:
DATASET_DIR

'./DaPhanLoai'